# LinkedIn Hard Mode
## Data cleaning

In [4]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('./data/LIHM post data.csv')

In [8]:
first_stats = df.loc[0, 'day_1']

In [19]:
stats_lst = [int(x.strip()) for x in first_stats.split(',')]

In [20]:
stats_lst

[2436, 21, 27]